## Create and run and end-to-end ML pipeline using AWS SageMaker and Lambda

**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if derivating this code for your own use-cases!**

### Step 0: Get Admin Setup Results
Bucket names, codecommit repo, docker image, IAM roles, ...

In order to keep things orginized, we will save our `Source Code` (data processing, model training/serving scripts), `datasets`, as well as our trained `model(s) binaries` and their `test-performance metrics` all on S3, **versioned with respect to the date/time of each update.**

In [ ]:
import sagemaker
import boto3
import zipfile
import json
from time import gmtime, strftime
from sagemaker.s3 import S3Uploader
session = boto3.session.Session()

# Grab admin resources (S3 Bucket name, IAM Roles and Docker Image for Training)
with open('admin_setup.txt', 'r') as filehandle:
    admin_setup = json.load(filehandle)

# MLOps Hygiene
BUCKET = admin_setup["project_bucket"]
SOURCE_DATA = admin_setup["raw_data_path"]
BRANCH = "master"
REPO = admin_setup["repo_name"]
WORKFLOW_NAME = admin_setup["workflow_name"]

REGION = session.region_name
TRAINING_IMAGE = admin_setup["docker_image"]
WORKFLOW_EXECUTION_ROLE = admin_setup["workflow_execution_role"]
WORKFLOW_DATE_TIME = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
SOURCE_CODE_PREFIX = "{}/{}".format(WORKFLOW_DATE_TIME, "source-code")

my_workflow_input = {
    #ADMIN
    "REGION":REGION,
    "ROLE_ARN":WORKFLOW_EXECUTION_ROLE,
    "BUCKET":BUCKET,
    "WORKFLOW_NAME":WORKFLOW_NAME,
    "WORKFLOW_DATE_TIME":WORKFLOW_DATE_TIME,
    "DATA_SOURCE":SOURCE_DATA,

    # CodeCommit
    "REPO":REPO,
    "BRANCH":BRANCH,
    "DATA_PROCESSING_DIR": "sagemaker-processing-src",
    "ML_DIR": "sagemaker-train-serve-src",
    
    # SM Processing
    "PROCESSING_SCRIPT":"processing.py",
    "PROCESSING_IMAGE":TRAINING_IMAGE,
    "PROCESSING_INSTANCE_TYPE":"ml.c5.xlarge",
    "PROCESSING_INSTANCE_COUNT":1,
    "PROCESSING_VOLUME_SIZE_GB":10,
    
    # SM TRAINING
    "TRAINING_SCRIPT":"train.py",
    "TRAINING_IMAGE":TRAINING_IMAGE,
    "TRAINING_INSTANCE_TYPE":"ml.c5.xlarge",
    "TRAINING_INSTANCE_COUNT":1,
    "TRAINING_VOLUME_SIZE_GB":10,
    
    # SM SERVING
    "SERVING_SCRIPT":"train.py",
    "SERVING_IMAGE":TRAINING_IMAGE,
    "SERVING_INSTANCE_TYPE":"ml.c5.xlarge",
    "SERVING_INSTANCE_COUNT":1,
    "SERVING_VOLUME_SIZE_GB":10,
}

# The following method will be used throughout this notebook to create Lambda functions without going to the console
session = sagemaker.Session()
lambda_client = boto3.client('lambda')

def create_lambda_function(zip_name, lambda_source_code, function_name, description):
    zf = zipfile.ZipFile(zip_name, mode='w')
    zf.write(lambda_source_code, arcname=lambda_source_code.split('/')[-1])
    zf.close()

    S3Uploader.upload(local_path=zip_name, 
                      desired_s3_uri="s3://{}/{}".format(BUCKET, SOURCE_CODE_PREFIX),
                      #session=session
                     )

    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.6',
        Role=WORKFLOW_EXECUTION_ROLE,
        Handler=zip_name.split('.')[0]+'.lambda_handler',
        Code={
            'S3Bucket': BUCKET,
            'S3Key': '{}/{}'.format(SOURCE_CODE_PREFIX, zip_name)
        },
        Description=description,
        Timeout=180,
        MemorySize=256
    )

### Step 1: Move Code from CodeCommit to S3
The first step in training a model on sagemaker is to copy our source code to S3. This step is automatically done for you when you use the SageMaker SDK.

In [ ]:
!pygmentize ./workflow-orchestration-src/codecommit_to_s3.py

Let's run the above script from a Lambda function, this will help us automoate this task later.

First create the Lambda function:

In [ ]:
create_lambda_function(zip_name="codecommit_to_s3.zip",
                       lambda_source_code="./workflow-orchestration-src/codecommit_to_s3.py",
                       function_name=WORKFLOW_NAME + '-codecommit-to-s3',
                       description="Copy code files from CodeCommit to a tarball on S3"
                      )

Run it:

In [ ]:
response = lambda_client.invoke(
    FunctionName=WORKFLOW_NAME + '-codecommit-to-s3',
    Payload=json.dumps(my_workflow_input).encode()
)

In [ ]:
response["Payload"].read()

In [ ]:
!aws s3 ls {"s3://{}/{}/source-code/".format(BUCKET, WORKFLOW_DATE_TIME)}

## Step 2: Run SageMaker Processing Job with `boto3`

The `boto3` client for SageMaker is more verbose than the SageMaker SDK yet gives more visibility in the low-level details of Amazon SageMaker.

To run a custom data processing script on SageMaker, we will use [boto3.client('sagemaker')
.create_processing_job()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_processing_job) inside a lambda function.

Here is the code for the lambda function:

In [ ]:
!pygmentize ./workflow-orchestration-src/create_sagemaker_prcoessing_job.py

Let's build it:

In [ ]:
create_lambda_function(zip_name="create_sagemaker_prcoessing_job.zip",
                       lambda_source_code="./workflow-orchestration-src/create_sagemaker_prcoessing_job.py",
                       function_name=WORKFLOW_NAME + '-create-sagemaker-prcoessing-job',
                       description="Creates Sagemaker Processing Job"
                      )

Run it:

In [ ]:
response = lambda_client.invoke(
    FunctionName=WORKFLOW_NAME + '-create-sagemaker-prcoessing-job',
    Payload=json.dumps(my_workflow_input).encode()
)

In [ ]:
response["Payload"].read()

Let's build a mechanism to check on the processing job status... again using a Lambda!

In [ ]:
create_lambda_function(zip_name='query_data_processing_status.zip',
                       lambda_source_code='./workflow-orchestration-src/query_data_processing_status.py',
                       function_name=WORKFLOW_NAME + '-query-data-processing-status',
                       description='Get Status of SageMaker Processing Job'
                      )

#### Make sure the SageMaker processing job is done (status = Completed) before luanching the training step

In [ ]:
import time
%cd ./workflow-orchestration-src
import query_data_processing_status as qs

while qs.lambda_handler(my_workflow_input, "")["ProcessingJobStatus"] == "InProgress":
    print(qs.lambda_handler(my_workflow_input, "")["ProcessingJobStatus"])
    time.sleep(5)

%cd ..

In [ ]:
!aws s3 ls {"s3://{}/{}/".format(BUCKET, WORKFLOW_DATE_TIME)}

## Step 3: Create SageMaker Training Job Using `boto3`

When using `boto3` to launch a training job, we must explicitly point it to our source code on S3 and docker image in addition to what SageMaker estimators expect.

Let's look at the code for the `create_sagemaker_training_job` lambda function.

In [ ]:
!pygmentize ./workflow-orchestration-src/create_sagemaker_training_job.py

Let's create it:

In [ ]:
create_lambda_function(zip_name='create_sagemaker_training_job.zip',
                       lambda_source_code='./workflow-orchestration-src/create_sagemaker_training_job.py',
                       function_name=WORKFLOW_NAME + '-create-sagemaker-training-job',
                       description='Creates SageMaker Training Job'
                      )

Run the training job once processing job is done:

In [ ]:
response = lambda_client.invoke(
    FunctionName=WORKFLOW_NAME + '-create-sagemaker-training-job',
    Payload=json.dumps(my_workflow_input).encode()
)

In [ ]:
response["Payload"].read()

Let's check on its status

In [ ]:
create_lambda_function(zip_name='query_training_status.zip',
                       lambda_source_code='./workflow-orchestration-src/query_training_status.py',
                       function_name=WORKFLOW_NAME + '-query-training-status',
                       description='Get Status of SageMaker Training Job'
                      )

#### Make sure the SageMaker training job is done (status = Completed) before deploying the model

In [ ]:
%cd ./workflow-orchestration-src

import query_training_status as qs

while qs.lambda_handler(my_workflow_input, "")["TrainingJobStatus"] == "InProgress":
    print(qs.lambda_handler(my_workflow_input, "")["TrainingJobStatus"])
    time.sleep(5)
    
%cd ..

In [ ]:
!aws s3 ls {"s3://{}/{}/".format(BUCKET, WORKFLOW_DATE_TIME)}

## Step 4: Deploy model on SageMaker using model artifacts on S3 using `boto3`

#### If training is done, then check model accuracy before deploying

In [ ]:
create_lambda_function(zip_name='query_model_accuracy.zip',
                       lambda_source_code='./workflow-orchestration-src/query_model_accuracy.py',
                       function_name=WORKFLOW_NAME + '-query-model-accuracy',
                       description='Get Model Accuracy from SageMaker Training Job'
                      )

In [ ]:
%cd ./workflow-orchestration-src

import query_model_accuracy as qs
print(qs.lambda_handler(my_workflow_input, ""))

%cd ..

Let's look at the code for the `deploy_sagemaker_model` lambda function. This function will be incharge of creating a SageMaker endpoint for our trained model. If endpoint exists, then it will update the endpoint with the new retrained model.

In [ ]:
!pygmentize ./workflow-orchestration-src/deploy_sagemaker_model.py

Again, let's put this function in a Lambda:

In [ ]:
create_lambda_function(zip_name='deploy_sagemaker_model.zip',
                       lambda_source_code='./workflow-orchestration-src/deploy_sagemaker_model.py',
                       function_name=WORKFLOW_NAME + '-deploy-sagemaker-model-job',
                       description='Creates and Deploys SageMaker Model From Training Artifacts'
                      )

And run it:

In [ ]:
response = lambda_client.invoke(
    FunctionName=WORKFLOW_NAME + '-deploy-sagemaker-model-job',
    Payload=json.dumps(my_workflow_input).encode()
)

In [ ]:
response["Payload"].read()

### Test Endpoint

In [ ]:
sagemaker_client = boto3.client('sagemaker')
endpoint_status = "Creating"
while endpoint_status != "InService":
    endpoint_status = sagemaker_client.describe_endpoint(**{'EndpointName': WORKFLOW_NAME})["EndpointStatus"]
    print(endpoint_status)
    time.sleep(5)

In [ ]:
import boto3
import pandas as pd
from sklearn.datasets import load_boston
import json
#session = sagemaker.Session()
data = load_boston()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['PRICE'] = data.target
print(df.shape)

sagemaker_runtime = boto3.client('sagemaker-runtime')
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=WORKFLOW_NAME,
    Body=df[data.feature_names].to_csv(header=False, index=False).encode('utf-8'),
    ContentType='text/csv')

decoded_response = json.loads(response['Body'].read().decode("utf-8"))
print(decoded_response[0:10])